## Downstream prediction using XGB

Perform downstream predictions using different embeddings.  Goal is to investigate the efficacy of the LSTM XGB pipeline in addition to comparing the self-supervised embedding outcomes (next and min) to the unsupervised (auto) and the supervised outcomes (hypo).

Train XGB models for hypoxemia, hypotension, and hypocapnia

In [ ]:
import os
from phase.prediction import *

PATH  = os.path.expanduser("~/phase/")
DPATH = PATH+"downstream_prediction/"
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"

lookback = 60
DEBUG = False

label_type_eta_currfeat_lst = [("desat_bool92_5_nodesat",0.02,"SAO2"),
                               ("nibpm60",0.1,"NIBPM"), 
                               ("etco235",0.1,"ETCO2")]

for label_type, eta, curr_feat in label_type_eta_currfeat_lst:
    print("\n[Progress] label_type: {}, eta: {}, curr_feat {}".format(label_type, eta, curr_feat))

    xgb_type = "xgb_{}_top15_eta{}".format(label_type,eta)
    RESDIR = '{}{}/'.format(RESULTPATH, xgb_type)
    if not os.path.exists(RESDIR): os.makedirs(RESDIR)

    for hosp_data in [0,1]:
        print("\n[Progress] hosp_data {}".format(hosp_data))
        dt_lst = ["raw[top15]+nonsignal",
                  "ema[top15]+nonsignal",
                  "randemb[top15]+nonsignal"]

        for hosp_model in [0,1]:
            print("\n[Progress] hosp_model {}".format(hosp_model))
            dt_lst += ["nextfive_{}[top15]+nonsignal".format(hosp_model),
                       "auto_{}[top15]+nonsignal".format(hosp_model),
                       "min5_{}[top15]+nonsignal".format(hosp_model)]
            
            if "desat" in label_type:
                dt_lst += ["hypox_{}[top15]+nonsignal".format(hosp_model)]
            elif "nibpm" in label_type:
                dt_lst += ["hypot_{}[top15]+nonsignal".format(hosp_model)]
            elif "etco2" in label_type:
                dt_lst += ["hypoc_{}[top15]+nonsignal".format(hosp_model)]
                
            for data_type in dt_lst:
                print("\n[Progress] data_type {}".format(data_type))

                (trainvalX,trainvalY) = load_data(DPATH,data_type,label_type,True,
                                                  hosp_data,curr_feat,DEBUG=DEBUG)
                print("[Progress] trainvalX.shape {}".format(trainvalX.shape))
                if not DEBUG:
                    train_xgb_model(RESDIR,trainvalX,trainvalY,"200epochs_"+data_type,
                                    label_type,hosp_data,eta)

                (test1X,test1Y)       = load_data(DPATH,data_type,label_type,False,
                                                  hosp_data,curr_feat,DEBUG=DEBUG)
                print("[Progress] test1X.shape    {}".format(test1X.shape))
                if not DEBUG:
                    load_xgb_model_and_test(RESDIR,test1X,test1Y,"200epochs_"+data_type,
                                            label_type,hosp_data,xgb_type,eta)

Train xgb on phenylephrine and hypertension

In [ ]:
import os
from phase.prediction import *

PATH  = os.path.expanduser("~/phase/")
DPATH = PATH+"downstream_prediction/"
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"

lookback = 60
DEBUG = False

label_type_eta_currfeat_lst = [("nibpm110",0.1,"NIBPM"), 
                               ("med_phenyl",0.1,"NIBPM")]

for label_type, eta, curr_feat in label_type_eta_currfeat_lst:
    print("\n*[Progress] label_type: {}, eta: {}, curr_feat {}".format(label_type, eta, curr_feat))

    xgb_type = "xgb_{}_top15_eta{}".format(label_type,eta)
    RESDIR = '{}{}/'.format(RESULTPATH, xgb_type)
    if not os.path.exists(RESDIR): os.makedirs(RESDIR)

    for hosp_data in [0,1]:
        print("\n***[Progress] hosp_data {}".format(hosp_data))

        dt_lst = ["randemb[top15]+nonsignal"]

        for hosp_model in [0,1]:
            dt_lst += ["nextfive_{}[top15]+nonsignal".format(hosp_model),
                       "auto_{}[top15]+nonsignal".format(hosp_model),
                       "min5_{}[top15]+nonsignal".format(hosp_model)]

        for data_type in dt_lst:
            print("\n*****[Progress] data_type {}".format(data_type))

            (trainvalX,trainvalY) = load_data(DPATH,data_type,label_type,True,hosp_data,curr_feat,DEBUG=DEBUG)
            print("[Progress] trainvalX.shape {}".format(trainvalX.shape))
            if not DEBUG:
                train_xgb_model(RESDIR,trainvalX,trainvalY2,"200epochs_"+data_type,label_type,hosp_data,eta)

            (test1X,test1Y)       = load_data(DPATH,data_type,label_type,False,hosp_data,curr_feat,DEBUG=DEBUG)
            print("[Progress] test1X.shape    {}".format(test1X.shape))
            if not DEBUG:
                load_xgb_model_and_test(RESDIR,test1X,test1Y2,"200epochs_"+data_type,label_type,hosp_data,xgb_type,eta)

Use large joint embedding model data (nextfivemulti)

In [ ]:
import os
from phase.prediction import *

PATH  = os.path.expanduser("~/phase/")
DPATH = PATH+"downstream_prediction/"
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"

lookback = 60
DEBUG = False

label_type_eta_currfeat_lst = [("desat_bool92_5_nodesat",0.02,"SAO2"),
                               ("nibpm60",0.1,"NIBPM"), 
                               ("etco235",0.1,"ETCO2")]

for label_type, eta, curr_feat in label_type_eta_currfeat_lst:
    print("\n[Progress] label_type: {}, eta: {}, curr_feat {}".format(label_type, eta, curr_feat))

    xgb_type = "xgb_{}_top15_eta{}".format(label_type,eta)
    RESDIR = '{}{}/'.format(RESULTPATH, xgb_type)
    if not os.path.exists(RESDIR): os.makedirs(RESDIR)

    for hosp_data in [0,1]:
        print("\n[Progress] hosp_data {}".format(hosp_data))

        for hosp_model in [0,1]:
            print("\n[Progress] hosp_model {}".format(hosp_model))
            dt_lst = ["nextfivemulti_{}[top15]+nonsignal".format(hosp_model)]
            
            for data_type in dt_lst:
                print("\n[Progress] data_type {}".format(data_type))

                (trainvalX,trainvalY) = load_data(DPATH,data_type,label_type,True,hosp_data,curr_feat,DEBUG=DEBUG)
                print("[Progress] trainvalX.shape {}".format(trainvalX.shape))
                if not DEBUG:
                    train_xgb_model(RESDIR,trainvalX,trainvalY,"200epochs_"+data_type,label_type,hosp_data,eta)

                (test1X,test1Y)       = load_data(DPATH,data_type,label_type,False,hosp_data,curr_feat,DEBUG=DEBUG)
                print("[Progress] test1X.shape    {}".format(test1X.shape))
                if not DEBUG:
                    load_xgb_model_and_test(RESDIR,test1X,test1Y,"200epochs_"+data_type,label_type,hosp_data,xgb_type,eta)